In [ ]:
import base64
import json
import re
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from elevenlabs.client import ElevenLabs
from elevenlabs.play import play

load_dotenv()

# -----------------------------
# Function Definitions
# -----------------------------

def encode_image_to_base64(image_path: str) -> str:
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def generate_json(img_base64: str) -> dict:
    messages = [
        {
            "role": "system",
            "content": (
                "You are an assistant. Analyze the given image and generate a short, neutral description. "
                "Return caption, tags, scam_type, and category in valid JSON format ONLY. "
                "Do NOT include any explanations, extra text, or emojis. "
                "The JSON must have keys: caption, tags, scam_type, category. "
                "If scam_type is not applicable, set it to 'Unknown'."
            )
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Describe this image."},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"}}
            ]
        }
    ]

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7
    )

    raw_output = response.choices[0].message.content
    match = re.search(r"\{.*\}", raw_output, re.DOTALL)
    json_text = match.group() if match else "{}"

    try:
        result = json.loads(json_text)
    except json.JSONDecodeError:
        result = {"caption": raw_output, "tags": [], "scam_type": "Unknown", "category": "Unknown"}

    # Ensure all keys exist
    result = {
        "caption": result.get("caption", ""),
        "tags": result.get("tags", []),
        "scam_type": result.get("scam_type", "Unknown"),
        "category": result.get("category", "Unknown")
    }
    
    return result

def generate_brainrot_image(caption: str, output_path="brainrot.png"):
    """
    Generate an Italian Brain Rot meme image in English with 3D rendered Italian Brain Rot characters.
    No bright colors, keep the scene coherent and humorous.
    No text from the script should appear on the image. If any text is present, it must be coherent, sharp, and not deformed.
    """
    prompt = (
        f"Create a humorous Italian Brain Rot meme scene in English. "
        f"Include 3D rendered characters juxtaposed together in absurd combinations. "
        f"Do NOT use bright colors; use muted, realistic tones. "
        f"The scene should visually reflect the caption/script: '{caption}', "
        f"but do NOT include any text from the script on the image. "
        f"If any text appears, it must be coherent, sharp, and not deformed. "
        f"Keep it visually coherent and funny, like a surreal meme, without chaotic elements."
    )
    client = OpenAI()

    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        response_format="b64_json"  # ensures we get base64 to save locally
    )

    image_b64 = response.data[0].b64_json
    with open(output_path, "wb") as f:
        f.write(base64.b64decode(image_b64))

    return output_path

def generate_meme_script(meme_json: dict) -> str:
    """
    Generates a meme script in the Italian Brain Rot style for an 8-second video.
    The script should be funny, absurd, and feature classic Brain Rot characters,
    but the scene should be visually coherent and not overly chaotic.
    """
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
    prompt = (
        "You are a meme scriptwriter specializing in the ITALIAN BRAIN ROT style.\n"
        "Your task is to write a script for a short, 8-second meme video.\n"
        "Requirements:\n"
        "- Use classic Italian Brain Rot meme characters and personalities.\n"
        "- Make the scene funny and absurd, but keep it visually coherent and easy to follow.\n"
        "- Include exaggerated reactions and ridiculous dialogue, but avoid excessive chaos or randomness.\n"
        "- Keep the style consistent with Italian Brain Rot memes.\n"
        "- Only return the script text, no JSON or metadata.\n"
        f"\nMeme details:\n{meme_json}\n"
    )
    response = llm.invoke(prompt)
    return response.content.strip()

def generate_meme_narration(script_text: str) -> str:
    """
    Converts a meme script into a funny, coherent narration for a 15-second video.
    The narration is energetic, uses variable expression tones, and humorous, but does not reference any characters, user, original prompt, or generate sound effects.
    """
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", 
         "You are a narration assistant for ITALIAN BRAIN ROT memes.\n"
         "Transform the given script into a humorous, energetic, and coherent narration for a 15-second video.\n"
         "Requirements:\n"
         "- Make the narration easy to follow and visually coherent.\n"
         "- Use variable expression tones to make the narration dynamic and engaging.\n"
         "- Add meme punchlines and absurd comments, but do not reference any characters, user, original prompt, or generate sound effects.\n"
         "- Do not include captions, emojis, formatting, or sound effects. Only plain, funny text."),
        ("user", "Convert this script into a narration in the Italian Brain Rot style:\n\n{script}")
    ])
    messages = prompt_template.format_messages(script=script_text)
    response = llm.invoke(messages)
    return response.content.strip()


def generate_and_play_audio(narration: str, voice_id: str = "pNInz6obpgDQGcFmaJgB"):
    elevenlabs = ElevenLabs(api_key=os.getenv("ELEVENLABS_API_KEY"))
    audio = elevenlabs.text_to_speech.convert(
        text=narration,
        voice_id=voice_id,
        model_id="eleven_multilingual_v2",
        output_format="mp3_44100_128",
    )
    play(audio)

# -----------------------------
# Main Pipeline
# -----------------------------
def main():
    img_base64 = encode_image_to_base64("user_upload.jpg")
    scam_json = generate_json(img_base64)
    print("✅ Scam JSON:")
    print(json.dumps(scam_json, indent=4))

    script_text = generate_meme_script(scam_json)
    print("\n🎬 Generated Script:\n", script_text)
    
    narration = generate_meme_narration(script_text)
    print("\n🎤 Narration Text for 11Labs:\n", narration)

    image_file = generate_brainrot_image(script_text)
    print("Generated Brain Rot image saved at:", image_file)
    
    generate_and_play_audio(narration)

if __name__ == "__main__":
    main()


✅ Scam JSON:
{
    "caption": "A text message alert from HSBC regarding unusual activity in a bank account, urging the recipient to visit a provided link.",
    "tags": [
        "HSBC",
        "bank",
        "text message",
        "scam",
        "phishing"
    ],
    "scam_type": "phishing",
    "category": "Fraud"
}

🎬 Generated Script:
 Scene:
[Interior of a chaotic Italian household, with classic Italian Brain Rot characters Nonna Maria and Luigi sitting at the kitchen table.]

Nonna Maria: *reading text message alert* "Eh?! Luigi, HSBC says there's unusual activity in our bank account! We must click this link, pronto!"

Luigi: *frantically waving hands* "No, Nonna! It's-a scam! Don't click, don't click!"

Nonna Maria: *ignoring Luigi and clicking the link* "Mamma mia, let's see what's inside!"

[Cut to a close-up of the computer screen showing a cartoonish scam message.]

Scam Message: "You win-a free spaghetti dinner! Just give us-a your bank details!"

[Nonna Maria's eyes wi

In [22]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import json

def generate_replay_script(scam_json: dict) -> str:
    """
    Generates an 8-12 second neutral, serious educational replay script based on the scam JSON.
    """
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    prompt_template = ChatPromptTemplate.from_messages([
        ("system",
         "You are an educational assistant. "
         "Generate a short, clear, and serious script explaining how a scam or misleading content works. "
         "Describe step-by-step actions and potential consequences in a factual and neutral tone. "
         "Keep it concise (8-12 seconds) and return ONLY the script text, no JSON, humor, or exaggeration."),
        ("user",
         "Use the following scam JSON to write the educational replay script:\n\n{scam_json}")
    ])

    messages = prompt_template.format_messages(scam_json=json.dumps(scam_json, indent=2))
    response = llm.invoke(messages)
    
    return response.content.strip()


def generate_narration_and_annotations(replay_script: str, scam_json: dict) -> dict:
    """
    Generates neutral narration text for TTS and factual visual annotations for the replay.
    Returns a dictionary with 'narration_text' and 'visual_annotations'.
    """
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    prompt_template = ChatPromptTemplate.from_messages([
        ("system",
         "You are a narration assistant for educational videos. "
         "Rewrite the replay script into clear, neutral, and serious narration suitable for TTS. "
         "Also generate visual annotation suggestions that highlight scam cues or manipulation points in a factual way. "
         "Return in JSON with keys: narration_text (string), visual_annotations (list of strings)."),
        ("user",
         "Replay Script:\n{replay_script}\n\nScam JSON:\n{scam_json}")
    ])
    
    messages = prompt_template.format_messages(
        replay_script=replay_script,
        scam_json=json.dumps(scam_json, indent=2)
    )
    
    response = llm.invoke(messages)
    
    # Attempt to parse JSON safely
    try:
        output = json.loads(response.content.strip())
    except json.JSONDecodeError:
        output = {
            "narration_text": response.content.strip(),
            "visual_annotations": []
        }
    
    return output

# Step 1: Generate replay script
replay_script = generate_replay_script(scam_json)
print("🎬 Replay Script:\n", replay_script)

# Step 2: Generate narration and visual annotations
narration_output = generate_narration_and_annotations(replay_script, scam_json)
print("🎤 Narration:\n", narration_output['narration_text'])
print("🖌️ Annotations:\n", narration_output['visual_annotations'])


NameError: name 'scam_json' is not defined

In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs
from elevenlabs.play import play
import os

load_dotenv()

# LLM setup
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Narration bot prompt
prompt_template = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a narration assistant. Rewrite scripts into smooth, human-like narration "
     "for a short 15-second video. Keep it conversational and easy for TTS. "
     "Do NOT include stage directions, emojis, or formatting. Just plain narration text."),
    ("user", "Convert this script into narration:\n\n{script}")
])

# Format and run
messages = prompt_template.format_messages(script=script_text)
response = llm.invoke(messages)

narration = response.content.strip()

print("🎤 Narration Text for 11Labs:\n", narration)

elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

audio = elevenlabs.text_to_speech.convert(
    text= narration ,
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    model_id="eleven_multilingual_v2",
    output_format="mp3_44100_128",
)

play(audio)

🎤 Narration Text for 11Labs:
 Cautious Citizen just got a text alert from HSBC asking for their details. With a sly smirk, they peek over their sunglasses and type carefully, saying, "Nice try, Phish Bond." Remember to stay sharp!


KeyboardInterrupt: 

In [ ]:
from runwayml import RunwayML

# The env var RUNWAYML_API_SECRET is expected to contain your API key.
client = RunwayML()

task = client.image_to_video.create(
  model='gen4_turbo',
  prompt_image='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTZouypNVCAMcyEZHJ_I1l1kiyut3KdrVvkTg&s',
  prompt_text='The bunny is eating a carrot',
  ratio='1280:720',
).wait_for_task_output()

print(task)

PermissionDeniedError: Error code: 403 - {'error': 'Model variant gen4_image is not available', 'docUrl': 'https://docs.dev.runwayml.com/api'}

In [55]:
text_entry = f"{result['caption']} | Tags: {', '.join(result['tags'])} | Character: {result['character']} | Scam Type: {result['scam_type']}"
print(text_entry)

When you get a text from your 'bank' but you haven't even opened an account yet. | Tags: scam, phishing, funny, textmessage | Character: Confused User | Scam Type: Phishing


In [49]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector = embeddings.embed_query(text_entry) 

In [53]:
from langchain.vectorstores import Chroma

# Initialize or load DB
db = Chroma(persist_directory="meme_db", embedding_function=embeddings)

# Add the new meme
db.add_texts([text_entry])
db.persist()

print("✅ Meme stored in vector DB")

✅ Meme stored in vector DB
